In [1]:
%cd ..

/home/rinjac/ondewo/ondewo-t2s-stella


In [2]:
from notebooks.nemo_modules import InferenceDataLayer
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.tts as nemo_tts
import wave
import numpy as np
from scipy.io.wavfile import write
import IPython

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2020-09-01 15:04:16 audio_preprocessing:61] Unable to import APEX. Mixed precision and distributed training will not work.
[NeMo W 2020-09-01 15:04:17 transformer_modules:36] Unable to import FusedLayerNorm  from APEX. Using regular LayerNorm instead.


In [3]:
neural_factory = nemo.core.NeuralModuleFactory(placement=nemo.core.DeviceType.GPU)

In [4]:
data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor.import_from_config(
        "notebooks/tacotron2.yaml", "AudioToMelSpectrogramPreprocessor"
    )

[NeMo I 2020-09-01 15:04:18 neural_modules:343] Loading configuration of a new Neural Module from the `notebooks/tacotron2.yaml` file
[NeMo I 2020-09-01 15:04:18 features:144] PADDING: 16
[NeMo I 2020-09-01 15:04:18 features:152] STFT using conv
[NeMo I 2020-09-01 15:04:20 neural_modules:443] Instantiated a new Neural Module named `audiotomelspectrogrampreprocessor0` of type `AudioToMelSpectrogramPreprocessor`


Create spectrogram

In [5]:
waveglow_conf = {
    'config-path': "models/waveglow/waveglow.yaml",
    'model-path': "models/waveglow/pretrained/WaveGlowNM.pt",
    'sigma': 0.6
}

In [6]:
waveglow = nemo_tts.WaveGlowInferNM.import_from_config(
    waveglow_conf['config-path'], "WaveGlowInferNM",
    overwrite_params={"sigma": waveglow_conf['sigma']}
)
waveglow.restore_from(waveglow_conf['model-path'])

[NeMo I 2020-09-01 15:04:20 neural_modules:343] Loading configuration of a new Neural Module from the `models/waveglow/waveglow.yaml` file
[NeMo I 2020-09-01 15:04:23 neural_modules:443] Instantiated a new Neural Module named `waveglowinfernm0` of type `WaveGlowInferNM`


In [7]:
waveglow.setup_denoiser()

In [15]:
afile = wave.open('notebooks/florian.wav')
samples = afile.getnframes()
audio = afile.readframes(samples)
audio_np_ = np.copy(np.frombuffer(audio, dtype=np.int16))
audio_np = audio_np_ / 2 ** 15
sample = audio_np

In [16]:
data_layer = InferenceDataLayer(sample=sample)
audio_signal, audio_signal_len = data_layer()
processed_signal, processed_signal_len = data_preprocessor(
    input_signal=audio_signal,
    length=audio_signal_len)
audio_pred = waveglow(mel_spectrogram=processed_signal)
audio_pred, mel_len_pred = neural_factory.infer(tensors=[audio_pred, processed_signal_len])

[NeMo I 2020-09-01 15:04:47 actions:695] Evaluating batch 0 out of 1


In [17]:
end_index: int = mel_len_pred[0].cpu().numpy()[0] * 256
audio = audio_pred[0].cpu().numpy()[0][:end_index]

In [18]:
audio, _ = waveglow.denoise(audio, strength=0.05)

In [19]:
save_file = "notebooks/florian_generated_denoise.wav"

In [20]:
# conversion to 16-bit PCM
audio *= 32768
audio = audio.astype("int16")

# save audio to file
write(save_file, 22050, audio)

In [21]:
IPython.display.Audio(save_file)